In [2]:

import os
import io
import base64
import requests
import json
from PIL import Image
from dotenv import load_dotenv, find_dotenv
import gradio as gr

# Load API keys from .env file
_ = load_dotenv(find_dotenv())
hf_api_key = os.environ['HF_API_KEY']

In [3]:
# Helper function to make API request and handle JSON/binary responses
def get_completion(inputs, parameters=None, ENDPOINT_URL=""):
    headers = {
        "Authorization": f"Bearer {hf_api_key}",
        "Content-Type": "application/json"
    }
    data = {"inputs": inputs}
    if parameters is not None:
        data.update({"parameters": parameters})
    
    response = requests.post(ENDPOINT_URL, headers=headers, data=json.dumps(data))
    
    # Check if response is JSON or binary content
    try:
        return response.json()
    except json.JSONDecodeError:
        # If binary, return the raw content
        return response.content

In [4]:
#######################################################
# Step 1: text-to-image
#######################################################

TTI_ENDPOINT = os.environ['HF_API_TTI_BASE']
ITT_ENDPOINT = os.environ['HF_API_ITT_BASE']

In [5]:
#######################################################
# Step 2: Building your game with `gr.Blocks()`
#######################################################

# In[ ]:


#######################################################
# Step 3: Bringing the functions from lessons 2 and 3!
#   - Lesson 2: Image Captioning 
#   - Lesson 3: Image Generation App!
#######################################################


# Function to convert PIL image to base64 string
def image_to_base64_str(pil_image):
    byte_arr = io.BytesIO()
    pil_image.save(byte_arr, format='PNG')
    byte_arr = byte_arr.getvalue()
    return base64.b64encode(byte_arr).decode('utf-8')

# Function to convert base64 string to PIL image
def base64_to_pil(img_base64):
    try:
        base64_decoded = base64.b64decode(img_base64)
        byte_stream = io.BytesIO(base64_decoded)
        pil_image = Image.open(byte_stream)
        return pil_image
    except (base64.binascii.Error, PIL.UnidentifiedImageError) as e:
        print("Error decoding image:", e)
        return None

# Image Captioning Function
def captioner(image):
    base64_image = image_to_base64_str(image)
    result = get_completion(base64_image, None, ITT_ENDPOINT)
    
    # Check if response is valid JSON and has 'generated_text' field
    if isinstance(result, list) and 'generated_text' in result[0]:
        return result[0]['generated_text']
    else:
        print("Error in captioning response:", result)
        return "Error generating caption."

# Image Generation Function
def generate(prompt):
    output = get_completion(prompt, None, TTI_ENDPOINT)
    
    # Check if the output is in JSON format or binary content
    if isinstance(output, bytes):
        try:
            # Directly interpret the binary response as an image
            byte_stream = io.BytesIO(output)
            result_image = Image.open(byte_stream)
            return result_image
        except (IOError, PIL.UnidentifiedImageError) as e:
            # Handle the error if the binary data is not a valid image
            print("Error interpreting binary image data:", e)
            return "Error: Unexpected binary data."
    elif isinstance(output, dict):
        # Check if the expected base64 image key is in the dictionary
        if 'generated_image_base64' in output:
            img_base64 = output['generated_image_base64']
            return base64_to_pil(img_base64)
        else:
            print("Error in image generation response:", output)
            return "Error generating image."
    else:
        print("Error: Unexpected response format.")
        return "Error: Unexpected response format from API."



In [6]:

#######################################################
# Step 4: First attempt, just captioning
#######################################################

# In[ ]:


import gradio as gr 
with gr.Blocks() as demo:
    gr.Markdown("# Describe-and-Generate game")
    image_upload = gr.Image(label=
            "Your first image",type="pil")
    btn_caption = gr.Button("Generate caption")
    caption = gr.Textbox(label="Generated caption")
    
    btn_caption.click(fn=captioner, 
          inputs=[image_upload], outputs=[caption])

gr.close_all()
demo.launch(share=True, server_port=
          int(os.environ['PORT1']))

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://d37224975ac78cfeee.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [7]:
#######################################################
# Step 5: Let's add generation
#######################################################

# In[ ]:


with gr.Blocks() as demo:
    gr.Markdown("# Describe-and-Generate game ")
    image_upload = gr.Image(label=
          "Your first image",type="pil")
    btn_caption = gr.Button("Generate caption")
    caption = gr.Textbox(label="Generated caption")
    btn_image = gr.Button("Generate image")
    image_output = gr.Image(label="Generated Image")
    btn_caption.click(fn=captioner, 
          inputs=[image_upload], outputs=[caption])
    btn_image.click(fn=generate, 
          inputs=[caption], outputs=[image_output])

gr.close_all()
demo.launch(share=True, 
          server_port=int(os.environ['PORT2']))

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://a4a69f7c3260671753.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [8]:
#######################################################
# Step 6: Doing it all at once! 
#        Lession 2: Image Captioning
#        Lession 3: Image Generation App
#######################################################

# In[ ]:


# Gradio interface
def caption_and_generate(image):
    caption = captioner(image)
    generated_image = generate(caption)
    return [caption, generated_image]

with gr.Blocks() as demo:
    gr.Markdown("# Describe-and-Generate game")
    image_upload = gr.Image(label="Your first image", type="pil")
    btn_all = gr.Button("Caption and generate")
    caption = gr.Textbox(label="Generated caption")
    image_output = gr.Image(label="Generated Image")

    btn_all.click(fn=caption_and_generate, inputs=[image_upload], outputs=[caption, image_output])

demo.launch(share=True, server_port=int(os.environ['PORT3']))


# In[ ]:


gr.close_all()

Running on local URL:  http://127.0.0.1:7863
Running on public URL: https://bb1f36b2a530ef101e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
